# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import time
from vaastav.fbref import *
from src.match_names import neutralize_name

# Getting raw data

In [3]:
# only Mo Salah
only_Salah = False
scrape_21_22 = False
scrape_22_23 = False
scrape_23_24 = True

In [4]:
def get_url( url ):
    df = pd.read_html( url, header=1)[0]
    return df

In [5]:
def get_shorter_season(season):
    res = season.split("-")
    return res[0] + "-" + res[1][2:]

In [6]:
def get_logs_for_players_in_season(players, season, players_names=[]):
    logs = pd.DataFrame({'A' : []})
    for id, player in players:
        player_name = neutralize_name(player)
        if only_Salah and player_name != "Mohamed-Salah": 
            continue
        print("Getting data for " + player_name)
        url = 'https://fbref.com/en/players/' + str(id) + '/matchlogs/' + season + '/summary/' + player_name + '-Match-Logs'
        try:
            new_player_df = get_url(url)
            new_player_df["Name"] = player_name
            new_player_df["Season"] = get_shorter_season(season)


            # dropping NaN rows
            new_player_df = new_player_df[new_player_df['Date'].notna()]

            # only Premier League
            # new_player_df = new_player_df[new_player_df.Comp == "Premier League"]

            # # only last 15 matches
            # new_player_df = new_player_df[-15:]

            if not logs.empty:
                logs = pd.concat([logs, new_player_df])
            else:
                logs = new_player_df
        except:
            print("Not found", player_name, season, id, url)
        
        # to not get timeouted
        time.sleep(3.2)
    return logs

In [7]:
tables = get_data("https://fbref.com/en/comps/9/wages/Premier-League-Wages")

In [8]:
# tables[1]

In [9]:
players = get_epl_players()

NOT FOUND Roméo Lavia
NOT FOUND Deivid Washington


In [10]:
len(players)

534

In [11]:
# players

In [12]:
# players['774cf58b'].data

In [13]:
def modify_df(df):
    df[['WDL', 'GoalsTeams']] = df['Result'].str.split(' ', n=1, expand=True)
    df[['Team Score', 'Opp Score']] = df['GoalsTeams'].str.split('–', expand=True)
    
    df = df.dropna(subset=['Team Score'])
    df = df.dropna(subset=['Opp Score'])
    
    df['Team Score'] = df['Team Score'].apply(lambda x: x.split(' ')[0].strip() if ' ' in x else x)
    df['Opp Score'] = df['Opp Score'].apply(lambda x: x.split(' ')[0].strip() if ' ' in x else x)

    df['Team Score'] = df['Team Score'].astype(int)
    df['Opp Score'] = df['Opp Score'].astype(int)
    df = df.drop(['Result'], axis=1)
    
    df = df.replace("On matchday squad, but did not play", 0)
    df["Min"] = df["Min"].astype(float)
    df["Gls"] = df["Gls"].astype(float)
    df["Ast"] = df["Ast"].astype(float)
    
    df["CS"] = (df["Opp Score"] == 0).astype(float)
    df["Was Home"] = (df["Venue"] == "Home").astype(float)
    df["GW"] = df['Round'].str.split(' ', expand=True)[1]
    
    df = df.sort_values(by=['Name', 'Date'], ascending=[True, True])
    
    return df.reset_index()

# 2021-22

In [14]:
%%time
# # player_names = ["Mohamed-Salah", "Miguel-Almiron", "Bukayo-Saka", "Kevin-De-Bruyne", "Wilfried-Zaha", "Mason-Mount", "Bruno-Fernandes"]
if scrape_21_22:
    logs_21_22 = get_logs_for_players_in_season(players, '2021-2022', None)
else:
    logs_21_22 = pd.read_csv("data/logs/logs_21_22.csv")

CPU times: total: 219 ms
Wall time: 396 ms


In [15]:
if scrape_21_22:
    logs_21_22 = modify_df(logs_21_22)
logs_21_22.shape

(21787, 57)

In [16]:
logs_21_22.to_csv("data/logs/logs_21_22.csv")

In [17]:
unwanted_clubs = ['Dynamo Kyiv', 'CSKA Moscow', 'Shakhtar']
logs_21_22_PL = logs_21_22[(logs_21_22["Comp"] == "Premier League") & (~logs_21_22["Squad"].isin(unwanted_clubs))]
logs_21_22_PL.to_csv("data/logs/logs_21_22_PL.csv")
logs_21_22_PL["Squad"].unique()

array(['Brighton', 'West Ham', 'Arsenal', 'Manchester Utd', 'Everton',
       'Liverpool', 'Newcastle Utd', 'Southampton', 'Manchester City',
       'Chelsea', 'Tottenham', 'Burnley', 'Aston Villa', 'Norwich City',
       'Wolves', 'Brentford', 'Crystal Palace', 'Watford',
       'Leicester City', 'Leeds United'], dtype=object)

# 2022-23

In [18]:
%%time
if scrape_22_23:
    logs_22_23 = get_logs_for_players_in_season(players, '2022-2023', None)
else:
    logs_22_23 = pd.read_csv("data/logs/logs_22_23.csv")

CPU times: total: 234 ms
Wall time: 455 ms


In [19]:
if scrape_22_23:
    logs_22_23 = modify_df(logs_22_23)
logs_22_23.shape

(21653, 57)

In [20]:
logs_22_23.to_csv("data/logs/logs_22_23.csv")

In [21]:
unwanted_clubs = ['Dynamo Kyiv', 'CSKA Moscow', 'Shakhtar']
logs_22_23_PL = logs_22_23[(logs_22_23["Comp"] == "Premier League") & (~logs_22_23["Squad"].isin(unwanted_clubs))]
logs_22_23_PL.to_csv("data/logs/logs_22_23_PL.csv")
logs_22_23_PL["Squad"].unique()

array(['West Ham', 'Brentford', 'Arsenal', 'Manchester Utd', 'Everton',
       'Brighton', 'Bournemouth', 'Liverpool', 'Crystal Palace', 'Fulham',
       'Aston Villa', 'Newcastle Utd', "Nott'ham Forest", 'Chelsea',
       'Tottenham', 'Manchester City', 'Wolves', 'Leicester City'],
      dtype=object)

# 2023-24

In [22]:
%%time
if scrape_23_24:
    logs_23_24 = get_logs_for_players_in_season(players, '2023-2024', None)
else:
    logs_23_24 = pd.read_csv("data/logs/logs_23_24.csv")

Getting data for Kevin-De-Bruyne
Getting data for Erling-Haaland
Getting data for Casemiro
Getting data for Jadon-Sancho
Getting data for Mohamed-Salah
Getting data for Raphael-Varane
Getting data for Raheem-Sterling
Getting data for Romelu-Lukaku
Not found Romelu-Lukaku 2023-2024 5eae500a https://fbref.com/en/players/5eae500a/matchlogs/2023-2024/summary/Romelu-Lukaku-Match-Logs
Getting data for Marcus-Rashford
Getting data for Bernardo-Silva
Getting data for Jack-Grealish
Getting data for Kai-Havertz
Getting data for Gabriel-Jesus
Getting data for Anthony-Martial
Getting data for Mason-Mount
Getting data for Joao-Cancelo
Not found Joao-Cancelo 2023-2024 bd6351cd https://fbref.com/en/players/bd6351cd/matchlogs/2023-2024/summary/Joao-Cancelo-Match-Logs
Getting data for John-Stones
Getting data for Reece-James
Getting data for Declan-Rice
Getting data for Bruno-Fernandes
Getting data for Phil-Foden
Getting data for Virgil-van-Dijk
Getting data for Rodri
Getting data for Sandro-Tonali
Get

In [23]:
if scrape_23_24:
    logs_23_24 = modify_df(logs_23_24)
logs_23_24.shape

(1224, 54)

In [24]:
logs_23_24.to_csv("data/logs_23_24.csv")

In [25]:
logs_23_24.tail()

,index,Date,Day,Comp,Round,Venue,Squad,Opponent,Start,Pos,...,OG,PKwon,PKcon,WDL,GoalsTeams,Team Score,Opp Score,CS,Was Home,GW
1219,2,2023-08-23,Wed,Conf Lg,Play-off round,Away,eng Aston Villa,sct Hibernian,N,NaN,...,NaN,NaN,NaN,W,5–0,5,0,1.0,0.0,round
1220,3,2023-08-27,Sun,Premier League,Matchweek 3,Away,Aston Villa,Burnley,N,CM,...,NaN,NaN,NaN,W,3–1,3,1,0.0,0.0,3
1221,0,2023-08-13,Sun,Premier League,Matchweek 1,Away,Tottenham,Brentford,Y,DM,...,NaN,NaN,NaN,D,2–2,2,2,0.0,0.0,1
1222,1,2023-08-19,Sat,Premier League,Matchweek 2,Home,Tottenham,Manchester Utd,Y,DM,...,NaN,NaN,NaN,W,2–0,2,0,1.0,1.0,2
1223,2,2023-08-26,Sat,Premier League,Matchweek 3,Away,Tottenham,Bournemouth,Y,DM,...,NaN,NaN,NaN,W,2–0,2,0,1.0,0.0,3


In [26]:
unwanted_clubs = ['Dynamo Kyiv', 'CSKA Moscow', 'Shakhtar']
logs_23_24_PL = logs_23_24[(logs_23_24["Comp"] == "Premier League") & (~logs_23_24["Squad"].isin(unwanted_clubs))]
logs_23_24_PL.to_csv("data/logs/logs_23_24_PL.csv")
logs_23_24_PL["Squad"].unique()

array(['West Ham', 'Brentford', 'Arsenal', 'Manchester Utd', 'Everton',
       'Sheffield Utd', 'Brighton', 'Fulham', 'Newcastle Utd',
       'Liverpool', 'Luton Town', 'Burnley', "Nott'ham Forest",
       'Bournemouth', 'Chelsea', 'Tottenham', 'Manchester City',
       'Aston Villa', 'Wolves', 'Crystal Palace'], dtype=object)

# Merged logs

In [27]:
# merging
logs = pd.concat([logs_21_22, logs_22_23, logs_23_24]).reset_index()
logs = logs.drop(columns=['level_0', 'index'])

In [28]:
logs.shape

(44664, 56)

In [29]:
logs.head(10)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Date,Day,Comp,Round,Venue,Squad,Opponent,...,OG,PKwon,PKcon,WDL,GoalsTeams,Team Score,Opp Score,CS,Was Home,GW
0,0.0,0.0,0.0,2021-08-21,Sat,Premier League,Matchweek 2,Home,Brighton,Watford,...,NaN,NaN,NaN,W,2–0,2,0,1.0,1.0,2
1,1.0,1.0,1.0,2021-08-28,Sat,Premier League,Matchweek 3,Home,Brighton,Everton,...,NaN,NaN,NaN,L,0–2,0,2,0.0,1.0,3
2,2.0,2.0,2.0,2021-09-01,Wed,WCQ,First round,Away,ie Rep. of Ireland,pt Portugal,...,NaN,NaN,NaN,L,1–2,1,2,0.0,0.0,round
3,3.0,3.0,3.0,2021-09-04,Sat,WCQ,First round,Home,ie Rep. of Ireland,az Azerbaijan,...,NaN,NaN,NaN,D,1–1,1,1,0.0,1.0,round
4,4.0,4.0,4.0,2021-09-19,Sun,Premier League,Matchweek 5,Home,Brighton,Leicester City,...,NaN,NaN,NaN,W,2–1,2,1,0.0,1.0,5
5,5.0,5.0,5.0,2021-09-22,Wed,EFL Cup,Third round,Home,Brighton,Swansea City,...,NaN,NaN,NaN,W,2–0,2,0,1.0,1.0,round
6,6.0,6.0,6.0,2021-09-27,Mon,Premier League,Matchweek 6,Away,Brighton,Crystal Palace,...,NaN,NaN,NaN,D,1–1,1,1,0.0,0.0,6
7,7.0,7.0,7.0,2021-10-02,Sat,Premier League,Matchweek 7,Home,Brighton,Arsenal,...,NaN,NaN,NaN,D,0–0,0,0,1.0,1.0,7
8,8.0,8.0,8.0,2021-10-09,Sat,WCQ,First round,Away,ie Rep. of Ireland,az Azerbaijan,...,NaN,NaN,NaN,W,3–0,3,0,1.0,0.0,round
9,9.0,9.0,9.0,2021-10-12,Tue,Friendlies (M),Friendlies (M),Home,ie Rep. of Ireland,qa Qatar,...,NaN,NaN,NaN,W,4–0,4,0,1.0,1.0,(M)


In [30]:
logs.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Date', 'Day', 'Comp',
       'Round', 'Venue', 'Squad', 'Opponent', 'Start', 'Pos', 'Min', 'Gls',
       'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Tkl',
       'Int', 'Blocks', 'xG', 'npxG', 'xAG', 'SCA', 'GCA', 'Cmp', 'Att',
       'Cmp%', 'PrgP', 'Carries', 'PrgC', 'Att.1', 'Succ', 'Match Report',
       'Name', 'Season', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'OG', 'PKwon',
       'PKcon', 'WDL', 'GoalsTeams', 'Team Score', 'Opp Score', 'CS',
       'Was Home', 'GW'],
      dtype='object')

In [31]:
logs.to_csv("data/logs_all.csv")

In [32]:
unwanted_clubs = ['Dynamo Kyiv', 'CSKA Moscow', 'Shakhtar']
logs_PL = logs[(logs["Comp"] == "Premier League") & (~logs["Squad"].isin(unwanted_clubs))]
logs_PL.to_csv("data/logs/logs_all_PL.csv")
logs_PL["Squad"].unique()

array(['Brighton', 'West Ham', 'Arsenal', 'Manchester Utd', 'Everton',
       'Liverpool', 'Newcastle Utd', 'Southampton', 'Manchester City',
       'Chelsea', 'Tottenham', 'Burnley', 'Aston Villa', 'Norwich City',
       'Wolves', 'Brentford', 'Crystal Palace', 'Watford',
       'Leicester City', 'Leeds United', 'Bournemouth', 'Fulham',
       "Nott'ham Forest", 'Sheffield Utd', 'Luton Town'], dtype=object)

# Features check

In [33]:
info = ["Date", "Day", "GW", "Was Home"]
features = ["Min", "Gls", "Sh", "SoT", "xG", "npxG", "xAG", "CS"]

In [34]:
logs[info + features]

,Date,Day,GW,Was Home,Min,Gls,Sh,SoT,xG,npxG,xAG,CS
0,2021-08-21,Sat,2,1.0,45.0,0.0,1.0,0.0,0.3,0.3,0.0,1.0
1,2021-08-28,Sat,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-09-01,Wed,round,0.0,71.0,0.0,4.0,0.0,NaN,NaN,NaN,0.0
3,2021-09-04,Sat,round,1.0,45.0,0.0,3.0,1.0,NaN,NaN,NaN,0.0
4,2021-09-19,Sun,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
44659,2023-08-23,Wed,round,0.0,24.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0
44660,2023-08-27,Sun,3,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44661,2023-08-13,Sun,1,0.0,90.0,0.0,3,0,0.0,0.0,0.0,0.0
44662,2023-08-19,Sat,2,1.0,90.0,0.0,2,0,0.1,0.1,0.0,1.0


In [35]:
logs[logs["Name"] == "Mohamed-Salah"]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Date,Day,Comp,Round,Venue,Squad,Opponent,...,OG,PKwon,PKcon,WDL,GoalsTeams,Team Score,Opp Score,CS,Was Home,GW
15383,15383.0,15383.0,15383.0,2021-08-14,Sat,Premier League,Matchweek 1,Away,Liverpool,Norwich City,...,NaN,NaN,NaN,W,3–0,3,0,1.0,0.0,1
15384,15384.0,15384.0,15384.0,2021-08-21,Sat,Premier League,Matchweek 2,Home,Liverpool,Burnley,...,NaN,NaN,NaN,W,2–0,2,0,1.0,1.0,2
15385,15385.0,15385.0,15385.0,2021-08-28,Sat,Premier League,Matchweek 3,Home,Liverpool,Chelsea,...,NaN,NaN,NaN,D,1–1,1,1,0.0,1.0,3
15386,15386.0,15386.0,15386.0,2021-09-05,Sun,WCQ,Second round,Away,eg Egypt,ga Gabon,...,NaN,NaN,NaN,D,1–1,1,1,0.0,0.0,round
15387,15387.0,15387.0,15387.0,2021-09-12,Sun,Premier League,Matchweek 4,Away,Liverpool,Leeds United,...,NaN,NaN,NaN,W,3–0,3,0,1.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37328,15541.0,15541.0,15541.0,2023-05-28,Sun,Premier League,Matchweek 38,Away,Liverpool,Southampton,...,NaN,NaN,NaN,D,4–4,4,4,0.0,0.0,38
37329,15542.0,15542.0,15542.0,2023-06-14,Wed,Africa Cup of Nations qualification,Group stage,Away,eg Egypt,gn Guinea,...,NaN,NaN,NaN,W,2–1,2,1,0.0,0.0,stage
44321,NaN,NaN,NaN,2023-08-13,Sun,Premier League,Matchweek 1,Away,Liverpool,Chelsea,...,NaN,NaN,NaN,D,1–1,1,1,0.0,0.0,1
44322,NaN,NaN,NaN,2023-08-19,Sat,Premier League,Matchweek 2,Home,Liverpool,Bournemouth,...,NaN,NaN,NaN,W,3–1,3,1,0.0,1.0,2
